In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import shutil
import time
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException

import warnings
from bs4 import BeautifulSoup

In [2]:
# 현재 작업 디렉토리를 가져오기
current_directory = os.getcwd()
# 상대경로 지정
relative_path = '..\ContentsList.xlsx'
# 상대경로를 절대경로로 변환
absolute_path = os.path.join(current_directory, relative_path)


In [14]:
# 'ContentsList.xlsx' 파일 불러오기
df = pd.read_excel(absolute_path)
df.columns

Index(['20240124', 'contents_url', 'category', 'board1', 'board2', 'title',
       'post_start', 'post_end'],
      dtype='object')

In [15]:
# Convert "post_start" to datetime with the specified format and then to 8-digit integer
df['post_start'] = pd.to_datetime(df['post_start'], format='%Y%m%d').dt.strftime('%Y%m%d').astype(int)

# Sorting by "post_start" and "node"
df.sort_values(by=["post_start", "node"], inplace=True)

# Adding a new column with today's date
today_date = datetime.today().strftime('%Y%m%d')
df.columns = [today_date] + list(df.columns[1:])

# Function to determine the prefix based on "node"
def get_prefix(node):
    if node.startswith("https://blog.naver.com/hanpro911"):
        return "BH"
    elif node.startswith("https://blog.naver.com/mysecondplace"):
        return "BM"
    elif node.startswith("https://blog.naver.com"):
        return "BZ"
    elif node.startswith("https://cafe.naver.com/mysecondhouse1"):
        return "CM"
    elif node.startswith("https://mysecondplace.co.kr") and ("bmode=write" in node or "prod_code" in node):
        return "HA"
    elif node.startswith("https://mysecondplace.co.kr"):
        return "HH"
    elif node.startswith("https://maily.so"):
        return "HB"
    elif node.startswith("https://whattime.co.kr"):
        return "HC"
    elif node.startswith("https://m2place.imweb.me"):
        return "HO"
    elif node.startswith("https://instagram.com/mysecondplace_"):
        return "|M"
    elif node.startswith("http://pf.kakao.com/_vxkxngb"):
        return "KC"
    elif node.startswith("http"):
        return "SE"
    else:
        return "SK"

# Counter
counter = {'BH': 1, 'BM': 1, 'BZ': 1, 'CM': 1, 'HA': 1, 'HH': 1, 'HB': 1, 'HC': 1, 'HO': 1, '|M': 1, 'KC': 1, 'SE': 1, 'SK': 1}

# Assigning values based on "node" conditions
for index, row in df.iterrows():
    prefix = get_prefix(row["node"])
    df.at[index, today_date] = f"{prefix}-{counter[prefix]:04d}"
    counter[prefix] += 1

# Sorting by "today_date"
df.sort_values(by=today_date, inplace=True)

# Save the DataFrame to a new variable or overwrite the existing DataFrame
sorted_df = df.copy()  # You can change the variable name if needed

In [16]:
sorted_df

,20240124,contents_url,category,board1,board2,title,post_start,post_end
0,BH-0001,https://blog.naver.com/hanpro911,blog_hanpro911,main,main,"구매하기, 마이세컨플레이스 (feat. 한프로) : 네이버 블로그",20220103,NaN
1,BH-0002,https://blog.naver.com/hanpro911/222611549543,blog_hanpro911,야매 DIY,야매 IOT,전원주택 방범 CCTV 설치하기 #1 (1/2.. : 네이버블로그,20220103,NaN
2,BH-0003,https://blog.naver.com/hanpro911/222611724607,blog_hanpro911,세컨하우스 개발,구매하기,세컨하우스용 시골집 매매하기 #1 (1/3) .. : 네이버블로그,20220103,NaN
3,BH-0004,https://blog.naver.com/hanpro911/222612556607,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #1 (1/.. : 네이버블로그,20220104,NaN
4,BH-0005,https://blog.naver.com/hanpro911/222612760199,blog_hanpro911,야매 DIY,노천탕 만들기,최신 로우테크 화목 노천탕 만들기 #2 (2/.. : 네이버블로그,20220104,NaN
...,...,...,...,...,...,...,...,...
112,HH-0059,https://mysecondplace.co.kr/ya0004,homepage,매물_양평,햇살가득2층집,양평 햇살 가득 2층집,20240119,NaN
113,HH-0060,https://mysecondplace.co.kr/ya0005,homepage,매물_양평,국유림산골짜기알땅,국유림 산골짜기 알땅,20240119,NaN
114,HH-0061,https://whattime.co.kr/msp_official/ya0003,homepage,매물_양평,목수아들이지은집_방문예약,양평 목수 아들이 어머니를 위해 지은 숲속집 - 마이세컨플레이스,20240119,NaN
115,HH-0062,https://whattime.co.kr/msp_official/ya0004,homepage,매물_양평,햇살가득2층집,양평 노을을 담은 집 - 마이세컨플레이스,20240119,NaN


# chrome -> url -> title

In [ ]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 없애기
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 브라우저 생성
browser = webdriver.Chrome(options=chrome_options)


# 타이틀을 저장할 리스트 생성
title_list = []

# date_list에 있는 각 URL을 방문하고 타이틀을 리스트에 추가
for url in sorted_df.contents_url:
    browser.get('https://blog.naver.com/hanpro911/223332459922')
    page_title = browser.title
    title_list.append(page_title)

In [13]:
title_list

['구매하기, 마이세컨플레이스 (feat. 한프로) : 네이버 블로그',
 '전원주택 방범 CCTV 설치하기 #1 (1/2.. : 네이버블로그',
 '세컨하우스용 시골집 매매하기 #1 (1/3) .. : 네이버블로그',
 '최신 로우테크 화목 노천탕 만들기 #1 (1/.. : 네이버블로그',
 '최신 로우테크 화목 노천탕 만들기 #2 (2/.. : 네이버블로그',
 '최신 로우테크 화목 노천탕 만들기 #3 (3/.. : 네이버블로그',
 '최신 로우테크 화목 노천탕 만들기 #4 (4/.. : 네이버블로그',
 '전원주택 방범 CCTV 설치하기 #2 (2/2.. : 네이버블로그',
 '세컨하우스용 시골집 매매하기 #2 (2/3) .. : 네이버블로그',
 '세컨하우스용 시골집 매매하기 #3 (3/3) .. : 네이버블로그',
 '세컨하우스용 시골집 매매하기 #번외편 (4/3.. : 네이버블로그',
 'EBS 한국기행 봄날의 촌캉스 제4부 촌으로 .. : 네이버블로그',
 '마이세컨플레이스란? : 네이버 블로그',
 '시골집을 리모델링한 충남 공주 세컨하우스 매매.. : 네이버블로그',
 '시골집을 리모델링한 충남 공주 유구읍 세컨하우.. : 네이버블로그',
 '전원주택 잔디관리, 로봇잔디깎기(가데나 실레노.. : 네이버블로그',
 '마이세컨플레이스가 호반의 도시 춘천으로 갑니다. : 네이버 블로그',
 '강원도 춘천에서 꺽지낚시를 해보자(feat. .. : 네이버블로그',
 '계곡에서 낚시로 잡은 꺽지구이를 먹어보자! : 네이버 블로그',
 '최신 로우테크 화목 노천탕 만들기 #5 (5/.. : 네이버블로그',
 '시골집 리모델링 어떻게 시작하죠?(마이세컨플레.. : 네이버블로그',
 '세컨하우스용 시골집 리모델링 방법 #0 (들어.. : 네이버블로그',
 '세컨하우스의 모든 것, 마이세컨플레이스 : 네이버 블로그',
 '우리에게 또 다른 집이 필요한 이유 : 네이버 블로그',
 '세컨하우스 뜻, 세컨하우스란 무엇일까? : 네이버 블로그',
 '듀얼라이프를 즐기

In [8]:
# assign the title list to 'title' column
sorted_df['title'] = title_list

In [17]:
result_df = sorted_df

In [18]:
result_df.to_excel(absolute_path, index=False)


In [ ]:

# Drop the original columns

all_values_series = pd.concat([
    referers_df['modified_url_before'],
    referers_df['modified_url_referer_1'],
    referers_df['modified_url_referer_2'],
    referers_df['modified_url_referer_3'],
    keyword_rows['before']
], ignore_index=True)

# 중복 제거
unique_values = all_values_series.drop_duplicates().sort_values(ascending=True)
unique_values = unique_values.dropna()
unique_values = unique_values.reset_index(drop=True)
unique_values_df = pd.DataFrame(unique_values)
unique_values_df = unique_values_df.rename(columns={unique_values_df.columns[0]: 'node'})

# Create a DataFrame from 'contentslist'
contents_df = pd.DataFrame(contentslist, columns=['node'])

# Merge 'unique_values_df' with 'contents_df' using a left join
merged_df = pd.merge(unique_values_df, contents_df, on='node', how='left', indicator=True)

# Filter rows where the value is only in 'unique_values_df'
missing_values = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis=1)
# Counter
prefix_counters = {'BH': 1, 'BM': 1, 'BZ': 1, 'CM': 1, 'HA': 1, 'HH': 1, 'HB': 1, 'HC': 1, 'HO': 1, '|M': 1, 'KC': 1, 'SE': 1, 'SK': 1}
today_date = datetime.today().strftime('%Y%m%d')

# Concatenate contentslist and missing_values vertically
df = pd.concat([contentslist, missing_values], ignore_index=True)
# Assign today_date if 'post_start' is empty
df['post_start'] = df['post_start'].fillna(today_date)


df.insert(1, today_date + "2", "")

# Assigning values based on "node" conditions
for index, row in df.iterrows():
    node_value = row["node"]
    prefix = get_prefix(node_value)

    # Get the last number for the current prefix from "contentslist"
    last_number = get_last_number(df.iloc[:, 0], prefix)

    # Create the column name with today's date and a potential suffix
    new_column_name = today_date + "2"

      # Assign the values based on conditions
    if pd.notna(node_value):  # Ensure node_value is not NaN
        df.at[index, "contentslist"] = f"{prefix}-{last_number + 1:04d}"
        df.loc[index, new_column_name] = f"{prefix}-{prefix_counters[prefix]:04d}"  # Update the existing column
        prefix_counters[prefix] += 1
    else:
        df.loc[index, new_column_name] = ""

df = df.drop(df.columns[0], axis=1)
df = df.drop(["contentslist"], axis=1)
df